In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = 

In [36]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia langchain_Huggingface

**Wikipedia retriver**

In [37]:
from langchain_community.retrievers import WikipediaRetriever

In [38]:
retriever=WikipediaRetriever(top_k_results=2,lang='en')

In [39]:
query="what is the story line of movie No Smoking release year=2007"
docs = retriever.invoke(query)
docs

[Document(metadata={'title': 'No Smoking (2007 film)', 'summary': 'No Smoking is a 2007 Indian Hindi-language surrealist psychological thriller film written and directed by Anurag Kashyap and co-produced by Vishal Bhardwaj and Kumar Mangat Pathak. The film stars John Abraham, Ayesha Takia, Ranvir Shorey and Paresh Rawal in the lead roles, while Bipasha Basu appears in an Item number. The film is loosely based upon the 1978 short story "Quitters, Inc." by Stephen King, which was previously adapted as one of three segments featured in the Hollywood anthology film, Cat\'s Eye (1985). It became the second Indian film after Julie Ganapathi and the first Hindi-language film to be adapted from Stephen King\'s work. The story follows K (Abraham), a self-obsessed, narcissist chain smoker who agrees to kick his habit to save his marriage and visits a rehabilitation centre, but is caught in a labyrinth game by Baba Bengali (Rawal), the man who guarantees he will make him quit.\nThe film released 

In [40]:
for i,doc in enumerate(docs):
  print(f"\n--- Result{i+1}---")
  print(f"Content:\n{doc.page_content}..")


--- Result1---
Content:
No Smoking is a 2007 Indian Hindi-language surrealist psychological thriller film written and directed by Anurag Kashyap and co-produced by Vishal Bhardwaj and Kumar Mangat Pathak. The film stars John Abraham, Ayesha Takia, Ranvir Shorey and Paresh Rawal in the lead roles, while Bipasha Basu appears in an Item number. The film is loosely based upon the 1978 short story "Quitters, Inc." by Stephen King, which was previously adapted as one of three segments featured in the Hollywood anthology film, Cat's Eye (1985). It became the second Indian film after Julie Ganapathi and the first Hindi-language film to be adapted from Stephen King's work. The story follows K (Abraham), a self-obsessed, narcissist chain smoker who agrees to kick his habit to save his marriage and visits a rehabilitation centre, but is caught in a labyrinth game by Baba Bengali (Rawal), the man who guarantees he will make him quit.
The film released worldwide on 26 October 2007, but was met wit

**Vector Store Retriever**

In [41]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_core.documents import Document

In [42]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

embedding model definition



In [44]:
embeddings = HuggingFaceEndpointEmbeddings(
    model="BAAI/bge-m3",
    task="feature-extraction",
    huggingfacehub_api_token=api_key,
)

In [45]:
vectorstore=Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="my_collection"
)

In [46]:
retriver=vectorstore.as_retriever(search_kwargs={'k':2})

In [47]:
query="what is Chroma used for?"
results=retriver.invoke(query)
print(results)

[Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'), Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.')]


In [48]:
for i ,doc in enumerate(results):
  print(f"\n--- Result {i+1} ---")
  print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Chroma is a vector database optimized for LLM-based search.


In [49]:
result=vectorstore.similarity_search(query,k=2)
for i,docs in enumerate(result):
  print(f"\n--- Result {i+1} ---")
  print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Chroma is a vector database optimized for LLM-based search.


**MMR** Maximum Marginal Retriever


In [50]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [51]:
from langchain_community.vectorstores import FAISS

In [52]:
vectorstore=FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)


In [53]:
retriever=vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3,"lambda_mult":0.5}
)

In [54]:
retriever.invoke("What is langchain?")

[Document(id='b6f0506e-4f77-4bbb-ab69-54416b543c9b', metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(id='b7ef6ba2-71b7-4886-a472-cf6cce7d408f', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'),
 Document(id='827e6de9-1c0a-45e2-b020-7e559b3086ba', metadata={}, page_content='Embeddings are vector representations of text.')]

## Multiquery Retriever

In [55]:
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [57]:
model = ChatOpenAI(
    model="nvidia/nemotron-nano-9b-v2:free",
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",  # Important: points to OpenRouter
    temperature=0.7,
    # max_tokens=400,
)

In [58]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [59]:
embeding_model=embeddings
vectorstore=FAISS.from_documents(
    documents=all_docs,
    embedding=embeding_model
)

In [60]:
similarity_search=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":5}
    )

In [61]:
from re import search
multiquery_retriever=MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={'k':5}),
    llm=model,
)

In [62]:
query="How to improve energy levels and maintain balance?"

In [63]:
similarity_result=similarity_search.invoke(query)
multiquery_result=multiquery_retriever.invoke(query)

In [64]:
for i, doc in enumerate(similarity_result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 5 ---
Regular walking boosts heart health and can reduce symptoms of depression.
******************************************************************************************************************************************************

--- Result 1 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 2 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 3 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and 

**ContextualCompressionRetriever**

In [65]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [66]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [67]:
vectorstore=FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)

In [73]:
base_retriever=vectorstore.as_retriever(search_kwargs={'k':5})

In [70]:
compressor=LLMChainExtractor.from_llm(model)

In [75]:
compression_retriever=ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [76]:
query="what is photosynthesis?"
compressed_results=compression_retriever.invoke(query)

In [77]:
for i,doc in enumerate(compressed_results):
  print(f"\n--- Result {i+1} ---")
  print(doc.page_content)


--- Result 1 ---
Extracted relevant parts:  
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
Photosynthesis does not occur in animal cells.

--- Result 3 ---
Extracted relevant parts:  
"The chlorophyll in plant cells captures sunlight during photosynthesis."
